In [1]:
### 1.数据接口
# encoding: UTF-8
import logging

logger = logging.getLogger()

import requests
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime

class SQI_DB(object):
    def __init__(self, url='http://182.254.241.204:5001', user='huangyangcheng@sqicapital.com', password='sq123456'):
        self.url = url
        self.user = user
        self.password = password
        self.token = self.generate_token()

    def generate_token(self):
        login_url = self.url + "/auth/login"
        headers = {'content-type': 'application/json'}
        login_info = {'email': self.user, 'password': self.password}
        login_response = requests.post(login_url, data=json.dumps(login_info), headers=headers)
        return json.loads(login_response.text)['auth_token']

    #-------------------------------------------------------------------------------------------------------------------
    #CCM日频时间序列接口
    def get_ssd(self, index_id, startday='', endday='', source='ifind', ret_type='df'):
        data_header = {'Authorization': 'Basic ' + self.token, 'content-type': 'application/json'}
        data_url = self.url + "/ssd"
        data_info = {
            'indexes': [
                {
                    'id': index_id,
                    'startday': startday,
                    'endday': endday,
                }
            ],
            'source': source
        }
        data_rsp = requests.post(data_url, data=json.dumps(data_info), headers=data_header)
        data = json.loads(json.loads(data_rsp.text)['data'])
        if ret_type == 'df':
            df = pd.DataFrame(data)
        else:
            # raise warning !
            raise ValueError('Unsupported ret_type={}'.format(ret_type))
        return df

    #CCM K线接口数据
    def get_bar(self, symbol, bartype, ret_type='df'):
        data_header = {'Authorization': 'Basic ' + self.token, 'content-type': 'application/json'}
        data_url = self.url + "/bar"
        data_info = {
            'index':
                {
                    'symbol': symbol,
                    'bartype': bartype  #'1 Min'  '5 Min' '15 Min'  '1 Hour' 'Daily'
                }
        }
        data_rsp = requests.post(data_url, data=json.dumps(data_info), headers=data_header)
        data = json.loads(json.loads(data_rsp.text)['data'])
        if ret_type == 'df':
            df = pd.DataFrame(data)
        else:
            raise ValueError('Unsupported ret_type={}'.format(ret_type))
        return df



In [2]:
download = SQI_DB()

In [17]:
download.get_ssd('ZN8888.SHF').dropna()

,tradingday,index_value,index_id
0,20070326,29175.0,ZN8888.SHF
1,20070327,29670.0,ZN8888.SHF
2,20070328,29523.0,ZN8888.SHF
3,20070329,29042.0,ZN8888.SHF
4,20070330,29389.0,ZN8888.SHF
...,...,...,...
3914,20230426,21205.0,ZN8888.SHF
3915,20230427,21047.0,ZN8888.SHF
3916,20230428,21197.0,ZN8888.SHF
3917,20230504,21026.0,ZN8888.SHF
